In [2]:
import os
import json
import shutil
import cv2

# 读取COCO-Hand-Big_annotations.txt文件
# 转换成COCO annotation格式
# 保存为COCO-Hand-Big_annotations.json文件
def convert2Coco():
    # 读取COCO-Hand-Big_annotations.txt文件
    with open('COCO-Hand-S_annotations.txt', 'r') as f:
        lines = f.readlines()
    # 读取图片文件夹
    img_dir = 'COCO-Hand-S_images'
    img_names = os.listdir(img_dir)
    # 初始化COCO annotation格式
    coco = {}
    coco['images'] = []
    coco['categories'] = []
    coco['annotations'] = []
    # 初始化categories
    category = {}
    category['id'] = 1
    category['name'] = 'hand'
    category['supercategory'] = 'hand'
    coco['categories'].append(category)
    # 初始化images
    for img_name in img_names:
        image = {}
        
        image['id'] = int(img_name.split('.')[0])
        image['file_name'] = img_name
        image['height'] = cv2.imread(img_dir + '/' + img_name).shape[0]
        image['width'] = cv2.imread(img_dir + '/' + img_name).shape[1]
        coco['images'].append(image)
    # 初始化annotations
    ann_id = 1
    for line in lines:
        line = line.strip().split(',')
        # 初始化annotation
        annotation = {}
        annotation['id'] = ann_id
        annotation['image_id'] = int(line[0].split('.')[0])
        annotation['category_id'] = 1
        annotation['iscrowd'] = 0
        # xmin=line[1] xmax=line[2] ymin=line[3] ymax=line[4]
        annotation['area'] = (int(line[2]) - int(line[1])) * (int(line[4]) - int(line[3]))
        annotation['bbox'] = [int(line[1]), int(line[3]), int(line[2]) - int(line[1]), int(line[4]) - int(line[3])]
        coco['annotations'].append(annotation)
        ann_id += 1
    # 保存为COCO-Hand-Big_annotations.json文件
    with open('COCO-Hand-S_annotations.json', 'w') as f:
        json.dump(coco, f)

# 划分训练集和验证集
# 保存为train.json和val.json文件
# 训练集：验证集 = 8 : 2
def splitTrainVal():
    # 读取COCO-Hand-Big_annotations.json文件
    with open('COCO-Hand-S_annotations.json', 'r') as f:
        coco = json.load(f)
    # 划分训练集和验证集
    train = {}
    val = {}
    train['images'] = []
    train['categories'] = coco['categories']
    train['annotations'] = []
    val['images'] = []
    val['categories'] = coco['categories']
    val['annotations'] = []
    for image in coco['images']:
        if image['id'] % 10 < 8:
            train['images'].append(image)
        else:
            val['images'].append(image)
    for annotation in coco['annotations']:
        if annotation['image_id'] % 10 < 8:
            train['annotations'].append(annotation)
        else:
            val['annotations'].append(annotation)
    # 保存为train.json和val.json文件
    with open('train.json', 'w') as f:
        json.dump(train, f)
    with open('val.json', 'w') as f:
        json.dump(val, f)


# 创建train和val文件夹
# 将训练集和验证集图片复制到train和val文件夹
# 创建annotations文件夹
# 将train.json和val.json移动到annotations文件夹
def constructFolder():
    # 创建train和val文件夹
    os.mkdir('train')
    os.mkdir('val')
    # 将训练集和验证集图片复制到train和val文件夹
    img_dir = 'COCO-Hand-S_images'
    img_names = os.listdir(img_dir)
    for img_name in img_names:
        if int(img_name.split('.')[0]) % 10 < 8:
            # python 拷贝文件 不使用os
            shutil.copyfile(img_dir + '/' + img_name, 'train/' + img_name)
        else:
            shutil.copyfile(img_dir + '/' + img_name, 'val/' + img_name)
    # 创建annotations文件夹
    os.mkdir('annotations')
    # 将train.json和val.json移动到annotations文件夹
    shutil.move('train.json', 'annotations/train.json')
    shutil.move('val.json', 'annotations/val.json')

In [3]:
convert2Coco()
splitTrainVal()
constructFolder()

In [12]:
# 在image_id=124456 的 image上画出bbox
import os
import json
import cv2
def drawBbox():
    # 读取COCO-Hand-Big_annotations.json文件
    with open('COCO-Hand-S_annotations.json', 'r') as f:
        coco = json.load(f)
    # 读取图片文件夹
    img_dir = 'COCO-Hand-S_images'
    img_names = os.listdir(img_dir)
    # 在image_id=124456 的 image上画出bbox
    for image in coco['images']:
        if image['id'] == 110:
            img_name = image['file_name']
            img = cv2.imread(img_dir + '/' + img_name)
            for annotation in coco['annotations']:
                if annotation['image_id'] == 110:
                    bbox = annotation['bbox']
                    cv2.rectangle(img, (bbox[0], bbox[1]), (bbox[0] + bbox[2], bbox[1] + bbox[3]), (0, 255, 0), 2)
            cv2.imwrite('124456.jpg', img)
drawBbox()